In [2]:
class Node(object):

    def __init__(self, parent=None):
        self.keys: list = []
        self.values: list[Node] = []
        self.parent: Node = parent

    def index(self, key):
        for i, item in enumerate(self.keys):
            if key < item:
                return i

        return len(self.keys)

    def __getitem__(self, item):
        return self.values[self.index(item)]

    def __setitem__(self, key, value):
        i = self.index(key)
        self.keys[i:i] = [key]
        self.values.pop(i)
        self.values[i:i] = value

    def split(self):
        left = Node(self.parent)

        mid = len(self.keys) // 2

        left.keys = self.keys[:mid]
        left.values = self.values[:mid + 1]
        for child in left.values:
            child.parent = left

        key = self.keys[mid]
        self.keys = self.keys[mid + 1:]
        self.values = self.values[mid + 1:]

        return key, [left, self]

    def __delitem__(self, key):
        i = self.index(key)
        del self.values[i]
        if i < len(self.keys):
            del self.keys[i]
        else:
            del self.keys[i - 1]

    def fusion(self):
        index = self.parent.index(self.keys[0])
        if index < len(self.parent.keys):
            next_node: Node = self.parent.values[index + 1]
            next_node.keys[0:0] = self.keys + [self.parent.keys[index]]
            for child in self.values:
                child.parent = next_node
            next_node.values[0:0] = self.values
        else:
            prev: Node = self.parent.values[-2]
            prev.keys += [self.parent.keys[-1]] + self.keys
            for child in self.values:
                child.parent = prev
            prev.values += self.values

    def borrow_key(self, minimum: int):
        index = self.parent.index(self.keys[0])
        if index < len(self.parent.keys):
            next_node: Node = self.parent.values[index + 1]
            if len(next_node.keys) > minimum:
                self.keys += [self.parent.keys[index]]

                borrow_node = next_node.values.pop(0)
                borrow_node.parent = self
                self.values += [borrow_node]
                self.parent.keys[index] = next_node.keys.pop(0)
                return True
        elif index != 0:
            prev: Node = self.parent.values[index - 1]
            if len(prev.keys) > minimum:
                self.keys[0:0] = [self.parent.keys[index - 1]]

                borrow_node = prev.values.pop()
                borrow_node.parent = self
                self.values[0:0] = [borrow_node]
                self.parent.keys[index - 1] = prev.keys.pop()
                return True

        return False


class Leaf(Node):
    def __init__(self, parent=None, prev_node=None, next_node=None):
        super(Leaf, self).__init__(parent)
        self.next: Leaf = next_node
        if next_node is not None:
            next_node.prev = self
        self.prev: Leaf = prev_node
        if prev_node is not None:
            prev_node.next = self

    def __getitem__(self, item):
        return self.values[self.keys.index(item)]

    def __setitem__(self, key, value):
        i = self.index(key)
        if key not in self.keys:
            self.keys[i:i] = [key]
            self.values[i:i] = [value]
        else:
            self.values[i - 1] = value

    def split(self):
        left = Leaf(self.parent, self.prev, self)
        mid = len(self.keys) // 2

        left.keys = self.keys[:mid]
        left.values = self.values[:mid]

        self.keys: list = self.keys[mid:]
        self.values: list = self.values[mid:]

        return self.keys[0], [left, self]

    def __delitem__(self, key):
        i = self.keys.index(key)
        del self.keys[i]
        del self.values[i]

    def fusion(self):
        if self.next is not None and self.next.parent == self.parent:
            self.next.keys[0:0] = self.keys
            self.next.values[0:0] = self.values
        else:
            self.prev.keys += self.keys
            self.prev.values += self.values

        if self.next is not None:
            self.next.prev = self.prev
        if self.prev is not None:
            self.prev.next = self.next

    def borrow_key(self, minimum: int):
        index = self.parent.index(self.keys[0])
        if index < len(self.parent.keys) and len(self.next.keys) > minimum:
            self.keys += [self.next.keys.pop(0)]
            self.values += [self.next.values.pop(0)]
            self.parent.keys[index] = self.next.keys[0]
            return True
        elif index != 0 and len(self.prev.keys) > minimum:
            self.keys[0:0] = [self.prev.keys.pop()]
            self.values[0:0] = [self.prev.values.pop()]
            self.parent.keys[index - 1] = self.keys[0]
            return True

        return False


class BPlusTree(object):
    root: Node

    def __init__(self, maximum=4):
        self.root = Leaf()
        self.maximum: int = maximum if maximum > 2 else 2
        self.minimum: int = maximum // 2
        self.depth = 0

    def find(self, key) -> Leaf:
        node = self.root

        while type(node) is not Leaf:
            node = node[key]
        return node

    def __getitem__(self, item):
        return self.find(item)[item]

    def query(self, key):
        leaf = self.find(key)
        return leaf[key] if key in leaf.keys else None

    def __setitem__(self, key, value, leaf=None):
        if leaf is None:
            leaf = self.find(key)
        leaf[key] = value
        if len(leaf.keys) > self.maximum:
            self.insert_index(*leaf.split())

    def insert_index(self, key, values: list[Node]):
        parent = values[1].parent
        if parent is None:
            values[0].parent = values[1].parent = self.root = Node()
            self.depth += 1
            self.root.keys = [key]
            self.root.values = values
            return

        parent[key] = values
        if len(parent.keys) > self.maximum:
            self.insert_index(*parent.split())


    def delete(self, key, node: Node = None):
        if node is None:
            node = self.find(key)
        del node[key]

        if len(node.keys) < self.minimum:
            if node == self.root:
                if len(self.root.keys) == 0 and len(self.root.values) > 0:
                    self.root = self.root.values[0]
                    self.root.parent = None
                    self.depth -= 1
                return

            elif not node.borrow_key(self.minimum):
                node.fusion()
                self.delete(key, node.parent)


    def show(self, node=None, file=None, _prefix="", _last=True):
        """Improved show method to display the tree structure."""
        if node is None:
            node = self.root
        if isinstance(node, str):
          return
        else:
          values = ', '.join(map(self.query, node.keys))
        print(_prefix, "└─ " if _last else "├─ ", values, sep="", file=file)
        _prefix += "   " if _last else "│  "

        if isinstance(node, Node):
            for i, child in enumerate(node.values):
                _last = (i == len(node.values) - 1)
                self.show(child, file, _prefix, _last)

    def leftmost_leaf(self) -> Leaf:
        """Improved leftmost_leaf method to find the leftmost leaf in the tree."""
        node = self.root
        while isinstance(node, Node) and not isinstance(node, Leaf):
            node = node.values[0]
        return node

    def query_more_than(self, key):
        """Improved query_more_than method to find all values greater than the given key."""
        results = []
        node = self.root

        # Find the leaf node where the key would be
        while not isinstance(node, Leaf):
            found = False
            for i, node_key in enumerate(node.keys):
                if key < node_key:
                    node = node.values[i]
                    found = True
                    break
            if not found:
                node = node.values[-1]

        # Collect all keys greater than the given key from the leaf nodes
        while node:
            for k, v in zip(node.keys, node.values):
                if k > key:
                    results.append(v)
            node = node.next
        return results

    def query_less_than(self, key):
        """Improved query_less_than method to find all values less than the given key."""
        results = []
        leaf = self.leftmost_leaf()

        # Collect all keys less than the given key from the leaf nodes
        while leaf:
            for k in leaf.keys:
                if k < key:
                    results.append(leaf[k])
                else:
                    return results
            leaf = leaf.next
        return results


def get_name_hash(name):
    letter_encoding = {
    'а': 1, 'б': 1, 'в': 2, 'г': 2, 'ґ': 2, 'д': 2, 'е': 2, 'є': 3, 'ж': 3, 'з': 3,
    'и': 4, 'і': 4, 'ї': 4, 'й': 4, 'к': 5, 'л': 5, 'м': 5, 'н': 5, 'о': 6, 'п': 6,
    'р': 6, 'с': 7, 'т': 7, 'у': 7, 'ф': 8, 'х': 8, 'ц': 8, 'ч': 8, 'ш': 9, 'щ': 9,
    'ь': 9, 'ю': 9, 'я': 9,
    }
    name = name.lower()
    hash_value = ''
    hash_len = 10

    # Додавання кодування початку слова
    start_part = name[:3]
    start_hash = ''.join(str(letter_encoding.get(char,'')) for char in start_part)
    hash_value += start_hash

    # Додавання кодування середини слова
    middle_part = name[3:]
    middle_hash = ''.join(str(letter_encoding.get(char, 0)) for char in middle_part)
    hash_value += middle_hash

    if len(hash_value) <= hash_len:
      hash_value += '0' * (hash_len - len(hash_value))
    else:
      hash_value = hash_value[:hash_len]

    # Додавання кодування довжини слова
    encoded_length = min(len(name), 10)  # обмежуємо довжину слова максимумом 10
    hash_value += str(encoded_length).zfill(2)

    return hash_value


def main():
  surnames = [
    "Шевченко", "Ковальчук", "Бойко", "Бондар", "Aсмаковець",
    "Змієвський", "Вертебний", "Олійник", "Симоненко", "Коновалець",
    "Боднарук", "Авраменко", "Сай", "Школа", "Тичина",
    "Мартинчик", "Мартиненко", "Дятел", "Литвиненко", "Пустова",
    "Куліш", "Компанієць", "Стременюк", "Федоренко",
    "Вовк", "Кузьменко", "Косогуб", "Гаврилюк", "Гончаренко",
    "Нестеренко", "Дядечко", "Шпак", "Стус", "Максименко",
    "Іващенко", "Юхно", "Шульга", "Шаповал", "Дмитренко",
    "Василюк", "Чередняк", "Білоус", "Косач",
    "Лісовий", "Довженко", "Кучма", "Короленко"
  ]
  surname_to_get = "Мартиненко"
  surname_to_hash = "Мартиненко"
  surname_to_more = "Чередняк"
  surname_to_less = "Дятел"
  surname_to_delete = "Пустова"

  bplustree = BPlusTree(4)
  for name in surnames:
    bplustree[get_name_hash(name)] = name
  bplustree.show()

  print(f'Перетворимо прізвище {surname_to_hash} за допомогою хеш функції, з завдання ЛР2:')
  print(get_name_hash(surname_to_hash))
  print(f'Дістанемо прізвище {surname_to_get} з дерева')
  print(bplustree.query(get_name_hash(surname_to_get)))

  # Пам'ятаємо, що в нас абетка з рівнозначними літерами зазначеними в ґеші, тому при виводі більших/менших порядок може бути цікавим
  print(f'Дістанемо прізвища, що більші за {surname_to_more} з дерева')
  print(bplustree.query_more_than(get_name_hash(surname_to_more)))

  print(f'Дістанемо прізвища, що менші за {surname_to_less} з дерева')
  print(bplustree.query_less_than(get_name_hash(surname_to_less)))


  print(f'Видалимо прізвище {surname_to_delete} з дерева')
  bplustree.delete(get_name_hash(surname_to_delete))
  bplustree.show()

if __name__ == '__main__':
  main()

└─ Гончаренко, Ковальчук, Симоненко
   ├─ Бондар, Вертебний
   │  ├─ Авраменко, Білоус, Боднарук, Бойко
   │  ├─ Бондар, Гаврилюк, Василюк
   │  └─ Вертебний, Дмитренко, Довженко, Вовк
   ├─ Змієвський, Мартинчик
   │  ├─ Гончаренко, Дядечко, Дятел
   │  ├─ Змієвський, Іващенко, Максименко, Мартиненко
   │  └─ Мартинчик, Нестеренко, Литвиненко, Лісовий
   ├─ Коновалець, Косогуб, Олійник
   │  ├─ Ковальчук, Компанієць
   │  ├─ Коновалець, Короленко, Косач
   │  ├─ Косогуб, Кузьменко, Куліш, Кучма
   │  └─ Олійник, Пустова, Сай
   └─ Aсмаковець, Федоренко, Школа
      ├─ Симоненко, Тичина
      ├─ Aсмаковець, Стременюк, Стус
      ├─ Федоренко, Чередняк, Шаповал, Шевченко
      └─ Школа, Шпак, Шульга, Юхно
Перетворимо прізвище Мартиненко за допомогою хеш функції, з завдання ЛР2:
516745255610
Дістанемо прізвище Мартиненко з дерева
Мартиненко
Дістанемо прізвища, що більші за Чередняк з дерева
['Шаповал', 'Шевченко', 'Школа', 'Шпак', 'Шульга', 'Юхно']
Дістанемо прізвища, що менші за Дятел з